![QuantConnect Logo](https://cdn.quantconnect.com/web/i/icon.png)
<hr>

In [1]:
# QuantBook Analysis Tool 
# For more information see [https://www.quantconnect.com/docs/research/overview]
qb = QuantBook()
spy = qb.AddEquity("SPY")
history = qb.History(qb.Securities.Keys, 360, Resolution.Daily)

# Indicator Analysis
bbdf = qb.Indicator(BollingerBands(30, 2), spy.Symbol, 360, Resolution.Daily)
bbdf.drop('standarddeviation', 1).plot()

In [2]:
    qb.SetStartDate(2015, 1, 1)  
    qb.SetEndDate(2020, 8, 1)  
    qb.SetCash(100000) 
    qb.AddEquity("SPY", Resolution.Daily)
#     qb.Schedule.On(self.DateRules.MonthStart("SPY"), self.TimeRules.AfterMarketOpen("SPY"), self.Rebalance)
#     qb.monthly_rebalance = False 
#     qb.filtered_fine = None
#     qb.UniverseSettings.Resolution = Resolution.Daily
#     qb.AddUniverse(self.CoarseSelectionFunction, self.FineSelectionFunction)
    
    def CoarseSelectionFunction(self, coarse):
        if self.monthly_rebalance:
            coarse = [x for x in coarse if (x.HasFundamentalData)
                                        and (x.Market == "usa")]
                                        
            return [i.Symbol for i in coarse]
        else:
            return []
    
    def FineSelectionFunction(self, fine):
        if self.monthly_rebalance:
            fine =[i for i in fine if ((i.SecurityReference.ExchangeId == "NYS") or (i.SecurityReference.ExchangeId == "ASE"))]
            self.filtered_fine = []

            for i in fine:
                i.MarketCap2 = float(i.EarningReports.BasicAverageShares.TwelveMonths * (i.EarningReports.BasicEPS.TwelveMonths*i.ValuationRatios.PERatio))
                history_start = self.History([i.Symbol], TimeSpan.FromDays(365))
                history_end = self.History([i.Symbol],TimeSpan.FromDays(335))
                if not history_start.empty and not history_end.empty:
                    i.Returns = float(history_end.iloc[0]["close"] - history_start.iloc[0]["close"])
                    self.filtered_fine.append(i)
            
            size = int(len(fine)*.3)
            self.filtered_fine = sorted(self.filtered_fine, key = lambda x: x.MarketCap2, reverse=True)
            self.filtered_fine = self.filtered_fine[:size]
            self.filtered_fine = sorted(self.filtered_fine, key = lambda x: x.Returns, reverse=True)
            symbols = [i.Symbol for i in self.filtered_fine]
            self.filtered_fine = symbols
            return symbols
        else:
            return []

In [3]:
qb.History
